### CNN 구현하기

In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [27]:
USE_CUDA = torch.cuda.is_available()

In [28]:
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [29]:
print(USE_CUDA)

False


### Hyperparameter 설정

In [30]:
EPOCHS     = 40
BATCH_SIZE = 64

### Data load

In [9]:
### transforms를 이용한 전처리는 파이토치 정규화, 텐서화만 적용
### transforms.ToTensor(), transforms.Normalize()

train_loader = torch.utils.data.DataLoader(
datasets.FashionMNIST('./.data',
                     train=True,
                     download=True,
                     transform = transforms.Compose([
                         transforms.ToTensor(),
                         transforms.Normalize((0.1307,) , (0.3081,))
                     ])),
batch_size = BATCH_SIZE, shuffle = True)

0it [00:00, ?it/s]

100%|█████████████████████████████████████████████████████████████████▉| 26411008/26421880 [01:17<00:00, 154795.62it/s]

Extracting ./.data\FashionMNIST\raw\train-images-idx3-ubyte.gz



0it [00:00, ?it/s]


  0%|                                                                                        | 0/29515 [00:00<?, ?it/s]
32768it [00:00, 34214.51it/s]                                                                                          

0it [00:00, ?it/s]

...


 96%|████████████████████████████████████████████████████████████████▌  | 4259840/4422102 [00:03<00:00, 1357631.08it/s]

0it [00:00, ?it/s]

Extracting ./.data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz




8192it [00:00, 13995.22it/s]                                                                                           


Extracting ./.data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz
Processing...
Done!


26427392it [01:30, 154795.62it/s]                                                                                      
4423680it [00:21, 1357631.08it/s]                                                                                      

In [11]:
test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                   train=False, 
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)

### CNN 구성하는 CLASS 생성하기

- 아래 CNN 모델은 
커널 크기는 5 x 5
컨볼루션 계층은 2개

- 파이토치의 nn.Conv2d 모듈은 입력 x를 받는 함수를 반환, 자신을 바로 부를 수 있는 인스턴스지만, 일단 함수로 생각해도 무방합니다. 
- 즉, self.conv1, self.conv2 와 같은 CNN 모델의 내부 변수들은 함수로 취급될 수 있습니다.

In [32]:
### Real version
### Super () - 자식 클래스에서 부모클래스의 내용을 사용하고 싶을경우 사용
### CNN은 텐서플로에서 nn.conv2d()라는 함수로 구현
# tf.nn.conv2d(input, filters, strides, padding)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # conv2d의 parameter는 입력 채널수 와 출력 채널수, 흑백이므로 입력 채널수에 색상이 1개 이므로 '1'
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        # 두번째 컨볼루션 계층에서 위 출력 10개의 특징맵을 받아서 20개의 특징맵을 생성
        # 커널 사이즈 지정 가능하고 1개의 숫자만 입력시 정사각형으로 인지
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        # nn.Dropout2d를 이용하여 드롭아웃 인스턴스 생성
        self.conv2_drop = nn.Dropout2d()
        
        # 일반 신경망 생성
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        # 두번째 입력은 커널 크기
        # F.max_pool2d()함수형이나 nn.MaxPool2d 일반 모듈을 사용해도 좋습니다.
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        # 일반 인공신경망은 1차원의 입력을 받으로 view를 통해서 차원변경 (-1, x가 가진 원소 갯수)
        x = x.view(-1, 320)
        # 활성화 함수
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)

In [25]:
### 커널 
class CNN(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size =5)
        self.conv2 = nn.Conv2d(10,20, kernel_size=5)
        self.drop = nn.Dropout2d()
        ### 일반 신경망
        self.fc1 = nn.Linear(320,50)
        self.fc2 = nn.Linear(50,10)
        
    def forward(self, x) :
        x = F.relu(F.max_pool2d(self.conv1(x),2))
        x = F.relu(F.max_pool2d(self.conv2(x),2))

In [23]:
# nn.Dropout2d 를 이용하여 컨볼루션 결과로 나올 출력값에 드롭아웃 적용
self.drop = nn.Dropout2d()

NameError: name 'self' is not defined

In [21]:
nn.Dropout2d()

Dropout2d(p=0.5)

In [ ]:
x = x.view(-1, 320)

### CNN 모델의 인스턴스와 최적화 함수 생성

In [33]:
model     = Net().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

### 모형학습

In [34]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        # F.cross_entropy() 오차함수 이용
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

### Test 성능확인

In [35]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 배치 오차를 합산
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()

            # 가장 높은 값을 가진 인덱스가 바로 예측값
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

### ResNET 구현

### Hyperparameter 

In [36]:
### 앞선 흑백보다 더 많이 해야합니다! (40,64)
EPOCHS     = 300
BATCH_SIZE = 128

In [37]:
### dataset 안에서 CIFAR10 으로만 변경해서 사용가능
### 과적합을 방지 하기 위해서 RandomCrop, RandomHorizontalFlop 노이즈를 추가

train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./.data',
                   train=True,
                   download=True,
                   transform=transforms.Compose([
                       transforms.RandomCrop(32, padding=4),
                       transforms.RandomHorizontalFlip(),
                       transforms.ToTensor(),
                       transforms.Normalize((0.5, 0.5, 0.5),
                                            (0.5, 0.5, 0.5))])),
    batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./.data',
                   train=False, 
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.5, 0.5, 0.5),
                                            (0.5, 0.5, 0.5))])),
    batch_size=BATCH_SIZE, shuffle=True)



0it [00:00, ?it/s]



  0%|                                                                                    | 0/170498071 [00:00<?, ?it/s]

  0%|                                                                    | 8192/170498071 [00:01<1:10:07, 40521.71it/s]

  0%|                                                                     | 40960/170498071 [00:01<54:38, 51989.77it/s]

  0%|                                                                    | 106496/170498071 [00:01<40:58, 69309.40it/s]

  0%|                                                                    | 155648/170498071 [00:01<32:16, 87954.66it/s]

  0%|                                                                   | 221184/170498071 [00:01<25:22, 111874.60it/s]

  0%|▏                                                                  | 401408/170498071 [00:02<18:47, 150881.76it/s]

  0%|▎                                                                  | 745472/170498071 [00:02<13:37, 207659.37it/s]

  1%|▍                        

 23%|██████████████▊                                                 | 39526400/170498071 [00:18<01:27, 1504995.15it/s]

 23%|██████████████▉                                                 | 39690240/170498071 [00:18<01:36, 1348746.08it/s]

 23%|██████████████▉                                                 | 39919616/170498071 [00:18<01:29, 1465700.86it/s]

 24%|███████████████                                                 | 40083456/170498071 [00:18<01:30, 1439991.70it/s]

 24%|███████████████                                                 | 40247296/170498071 [00:19<01:32, 1402051.06it/s]

 24%|███████████████▏                                                | 40460288/170498071 [00:19<01:24, 1537522.42it/s]

 24%|███████████████▏                                                | 40624128/170498071 [00:19<01:34, 1376723.63it/s]

 24%|███████████████▎                                                | 40869888/170498071 [00:19<01:27, 1479875.63it/s]

 24%|███████████████▍           

 35%|██████████████████████▌                                         | 60170240/170498071 [00:39<01:06, 1664930.00it/s]

 35%|██████████████████████▋                                         | 60366848/170498071 [00:39<01:11, 1534984.66it/s]

 36%|██████████████████████▊                                         | 60612608/170498071 [00:39<01:11, 1529650.08it/s]

 36%|██████████████████████▉                                         | 60956672/170498071 [00:39<00:59, 1832968.86it/s]

 36%|██████████████████████▉                                         | 61177856/170498071 [00:40<01:05, 1678890.84it/s]

 36%|███████████████████████                                         | 61448192/170498071 [00:40<01:05, 1669056.44it/s]

 36%|███████████████████████▏                                        | 61636608/170498071 [00:40<01:30, 1199498.42it/s]

 37%|███████████████████████▎                                        | 62267392/170498071 [00:40<01:11, 1512156.87it/s]

 37%|███████████████████████▍   

 47%|█████████████████████████████▉                                  | 79798272/170498071 [00:58<01:27, 1034119.34it/s]

 47%|██████████████████████████████▍                                  | 79921152/170498071 [00:58<01:36, 934899.33it/s]

 47%|██████████████████████████████▌                                  | 80060416/170498071 [00:58<01:37, 924249.63it/s]

 47%|██████████████████████████████▏                                 | 80257024/170498071 [00:59<01:23, 1076029.25it/s]

 47%|██████████████████████████████▋                                  | 80388096/170498071 [00:59<01:30, 993509.50it/s]

 47%|██████████████████████████████▋                                  | 80535552/170498071 [00:59<01:31, 981029.57it/s]

 47%|██████████████████████████████▎                                 | 80732160/170498071 [00:59<01:19, 1130981.39it/s]

 47%|██████████████████████████████▎                                 | 80863232/170498071 [00:59<01:27, 1029802.09it/s]

 48%|███████████████████████████

 56%|████████████████████████████████████▍                            | 95625216/170498071 [01:16<01:55, 646930.82it/s]

 56%|████████████████████████████████████▌                            | 95756288/170498071 [01:16<01:57, 638715.68it/s]

 56%|████████████████████████████████████▌                            | 95887360/170498071 [01:16<01:46, 698161.58it/s]

 56%|████████████████████████████████████▌                            | 95969280/170498071 [01:16<01:46, 697146.74it/s]

 56%|████████████████████████████████████▌                            | 96051200/170498071 [01:17<01:58, 630596.76it/s]

 56%|████████████████████████████████████▋                            | 96182272/170498071 [01:17<01:47, 693572.40it/s]

 56%|████████████████████████████████████▋                            | 96264192/170498071 [01:17<01:47, 693217.39it/s]

 57%|████████████████████████████████████▋                            | 96346112/170498071 [01:17<01:57, 630407.74it/s]

 57%|███████████████████████████

 68%|███████████████████████████████████████████▌                    | 116023296/170498071 [01:36<01:02, 871441.92it/s]

 68%|███████████████████████████████████████████▌                    | 116203520/170498071 [01:36<01:02, 866394.53it/s]

 68%|███████████████████████████████████████████▋                    | 116383744/170498071 [01:37<01:02, 865718.43it/s]

 68%|███████████████████████████████████████████▊                    | 116563968/170498071 [01:37<00:58, 918713.87it/s]

 68%|███████████████████████████████████████████                    | 116695040/170498071 [01:37<00:53, 1002943.52it/s]

 69%|███████████████████████████████████████████▊                    | 116809728/170498071 [01:37<01:01, 866679.22it/s]

 69%|███████████████████████████████████████████▉                    | 116957184/170498071 [01:37<01:03, 837327.04it/s]

 69%|███████████████████████████████████████████▉                    | 117137408/170498071 [01:37<00:54, 973355.39it/s]

 69%|███████████████████████████

 76%|████████████████████████████████████████████████▊               | 130146304/170498071 [01:54<00:50, 806405.64it/s]

 76%|████████████████████████████████████████████████▉               | 130236416/170498071 [01:55<00:51, 778026.08it/s]

 76%|████████████████████████████████████████████████▉               | 130326528/170498071 [01:55<00:51, 773064.76it/s]

 76%|████████████████████████████████████████████████▉               | 130408448/170498071 [01:55<00:51, 771652.40it/s]

 77%|████████████████████████████████████████████████▉               | 130506752/170498071 [01:55<00:55, 715171.38it/s]

 77%|█████████████████████████████████████████████████               | 130654208/170498071 [01:55<00:48, 825096.89it/s]

 77%|█████████████████████████████████████████████████               | 130752512/170498071 [01:55<00:51, 778068.16it/s]

 77%|█████████████████████████████████████████████████               | 130850816/170498071 [01:55<00:48, 815705.05it/s]

 77%|███████████████████████████

 82%|████████████████████████████████████████████████████▏           | 139182080/170498071 [02:12<01:06, 468587.28it/s]

 82%|████████████████████████████████████████████████████▎           | 139239424/170498071 [02:12<01:03, 494795.08it/s]

 82%|████████████████████████████████████████████████████▎           | 139296768/170498071 [02:12<01:02, 500756.77it/s]

 82%|████████████████████████████████████████████████████▎           | 139354112/170498071 [02:13<01:12, 431145.12it/s]

 82%|████████████████████████████████████████████████████▎           | 139419648/170498071 [02:13<01:06, 468907.48it/s]

 82%|████████████████████████████████████████████████████▎           | 139476992/170498071 [02:13<01:07, 456842.35it/s]

 82%|████████████████████████████████████████████████████▍           | 139550720/170498071 [02:13<01:04, 481340.80it/s]

 82%|████████████████████████████████████████████████████▍           | 139608064/170498071 [02:13<01:04, 476711.31it/s]

 82%|███████████████████████████

 93%|██████████████████████████████████████████████████████████▎    | 157933568/170498071 [02:28<00:08, 1443289.41it/s]

 93%|██████████████████████████████████████████████████████████▍    | 158089216/170498071 [02:28<00:08, 1441510.56it/s]

 93%|██████████████████████████████████████████████████████████▍    | 158244864/170498071 [02:28<00:10, 1201203.97it/s]

 93%|██████████████████████████████████████████████████████████▌    | 158490624/170498071 [02:29<00:08, 1396333.11it/s]

 93%|██████████████████████████████████████████████████████████▌    | 158654464/170498071 [02:29<00:08, 1362717.32it/s]

 93%|██████████████████████████████████████████████████████████▋    | 158834688/170498071 [02:29<00:08, 1447915.53it/s]

 93%|██████████████████████████████████████████████████████████▊    | 158998528/170498071 [02:29<00:08, 1422461.64it/s]

 93%|██████████████████████████████████████████████████████████▊    | 159154176/170498071 [02:29<00:09, 1222383.54it/s]

 93%|███████████████████████████

### ResNet 구현하기

- Residual 블록을 BasicBlock 이라는 새로운 파이토치 모듈로 정의해서 사용
- 파이토치에서는 nn.Module을 이용하여 모듈 위에 또 다른 모듈을 쌓아 올릴 수 있음
##### nn.BatchNorm2d 새로운 계층 사용
- 배치 정규화를 수행하는 계층으로, 학습률을 너무 높게 잡으면 기울기 소실이 되는 현상이 발생하는데, 이를 예빵하고 학습과정을 안정화하기 위해 학습중 각 계층에 들어가는 입력을 평균과 분산을 정규화 시킴. 배치 정규화는 신경망 내부 데이터에 직접 영향을 주는 방식이고 드롭아웃은 학습 중에서 데이터 일부를 배제하여 간접적으로 과적합을 막는 방식

In [55]:
### Residual을 만들자! Basic Block Code

class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        
        ### self.bn2 계층의 출력 크기와 같은 planes 와 더해주는 selft.shortcut 모듈 정의
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        ### nn.Sequential은 여러 모듈을 하나의 모듈로 묶는 역할. 
        ### output x를 self.shortcut을 거치게 하여 크기가 같게하고 활성화 함수를 거칫 값에 더해줌
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [ ]:
class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()
        
        ### layer1 층이 입력받는 채널의 갯수가 16 이므로 16으로 초기화
        self.in_planes = 16

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3,adding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        
        self.layer1 = self._make_layer(16, 2, stride=1)
        self.layer2 = self._make_layer(32, 2, stride=2)
        self.layer3 = self._make_layer(64, 2, stride=2)
        self.linear = nn.Linear(64, num_classes)

        self.layer2 = self._make_layer(32, 2, stride=2)
        self.layer3 = self._make_layer(64, 2, stride=2)
        self.linear = nn.Linear(64, num_classes)

    ### self._make_layer() : 파이토치의 nn.Suquential 도구로 여러 BasicBlock 을 모듈 하나로 묶어주는 역할  
    ### 증폭하는 역할을 하는 모듈들은 shortcut모듈을 따로 가지게 됩니다
    def _make_layer(self, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(BasicBlock(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        # 평균풀링?
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

### 학습률 감소 기법 사용

In [57]:
model = ResNet().to(DEVICE)

optimizer = optim.SGD(model.parameters(), lr=0.1,
                      momentum=0.9, weight_decay=0.0005)

### 파이토치 내부의 optim.lr_scheduler.StepLR 도구를 사용해 간단하게 적용 가능
### Scheduler는 이폭마다 호출되며 step_size 를 50으로 지정해주어 50번 호출 될 때 0.1를 곱해서 0.1로 시작하면 50 이폭 후에 0.01

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)

In [58]:
print(model)

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=

### 학습하기

In [59]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

### 테스트하기

In [60]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 배치 오차를 합산
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()

            # 가장 높은 값을 가진 인덱스가 바로 예측값
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

### 수행

In [ ]:
### scheduler.step() 함수로 학습률을 조금 낮춰주는 단계가 추가

for epoch in range(1, EPOCHS + 1):
    scheduler.step()
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

[1] Test Loss: 1.6520, Accuracy: 42.64%
[2] Test Loss: 1.2980, Accuracy: 55.94%
[3] Test Loss: 1.0338, Accuracy: 64.34%
[4] Test Loss: 0.9612, Accuracy: 67.65%
[5] Test Loss: 1.0750, Accuracy: 65.33%
[6] Test Loss: 0.8132, Accuracy: 71.26%
[7] Test Loss: 0.8852, Accuracy: 70.79%
[8] Test Loss: 0.6904, Accuracy: 76.27%
[9] Test Loss: 0.7184, Accuracy: 76.05%
[10] Test Loss: 0.7912, Accuracy: 74.09%
[11] Test Loss: 1.0133, Accuracy: 67.83%
[12] Test Loss: 0.6257, Accuracy: 78.30%
[13] Test Loss: 0.8413, Accuracy: 73.28%
[14] Test Loss: 0.7385, Accuracy: 75.12%
[15] Test Loss: 0.7158, Accuracy: 76.45%
[16] Test Loss: 0.9716, Accuracy: 69.27%
[17] Test Loss: 0.6659, Accuracy: 77.77%
[18] Test Loss: 0.9226, Accuracy: 71.32%
